<a href="https://colab.research.google.com/github/stevanmatovic/serbian-sentiment-analysis/blob/master/BERT_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.71.42.234:8470

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 1099171824860403783),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 4499872631423793810),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 5000302068286273353),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 12418468321052546360),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 3143836248516840034),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 4677492730179860369),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 1717

In [2]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub 
import tensorflow_hub as hub

Cloning into 'bert_repo'...
remote: Enumerating objects: 325, done.
remote: Total 325 (delta 0), reused 0 (delta 0), pack-reused 325
Receiving objects: 100% (325/325), 258.56 KiB | 3.54 MiB/s, done.
Resolving deltas: 100% (184/184), done.


W0404 12:59:33.356117 139951474530176 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [3]:
BUCKET = 'stevan-bucket'
OUTPUT_DIR = 'gs://{}/bert-tfhub/models/donesi'.format(BUCKET)
# tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: gs://stevan-bucket/bert-tfhub/models/donesi *****


In [0]:
BERT_MODEL = 'multi_cased_L-12_H-768_A-12' 
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

In [5]:
tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)

Instructions for updating:
Colocations handled automatically by placer.


W0404 12:59:45.336371 139951474530176 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0404 12:59:47.763240 139951474530176 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [6]:
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from run_classifier import InputExample

drive.mount('/content/drive')

df = pd.read_csv('/content/drive/My Drive/raw.tsv',sep='\t')

def get_examples(dataframe):
    """Creates examples for the training and dev sets."""
    
    train_df, test_df = train_test_split(dataframe,test_size=0.2, random_state=4)
    train = []
    test = []
    for i, row in train_df.iterrows():
      guid = "train-%s" % (i)
      text_a = tokenization.convert_to_unicode(row['Review'])
      label = tokenization.convert_to_unicode(str(row['Label']))
      train.append(InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    for i, row in test_df.iterrows():
      guid = "test-%s" % ( i)
      text_a = tokenization.convert_to_unicode(row['Review'])
      label = tokenization.convert_to_unicode(str(row['Label']))
      test.append(InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    return train, test
  
train, test = get_examples(df)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
MAX_SEQ_LENGTH = 512
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

label_list =["0","1"]

# Compute number of train and warmup steps from batch size
num_train_steps = int(len(train) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION) 

# Setup TPU related config
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
ITERATIONS_PER_LOOP = 1000

def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

In [8]:
# Force TF Hub writes to the GS bucket we provide.
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR

model_fn = run_classifier_with_tfhub.model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  bert_hub_module_handle=BERT_MODEL_HUB
)

estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)

W0404 13:08:51.875296 139951474530176 estimator.py:1924] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f48b0cea0d0>) includes params argument, but params are not passed to Estimator.


INFO:tensorflow:Using config: {'_model_dir': 'gs://stevan-bucket/bert-tfhub/models/donesi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.71.42.234:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f48b0cefef0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.71.42.234:8470', '_evaluation_master': 'grpc://10.71.42.234:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num

I0404 13:08:51.882212 139951474530176 estimator.py:201] Using config: {'_model_dir': 'gs://stevan-bucket/bert-tfhub/models/donesi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.71.42.234:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f48b0cefef0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.71.42.234:8470', '_evaluation_master': 'grpc://10.71.42.234:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(ite

INFO:tensorflow:_TPUContext: eval_on_tpu True


I0404 13:08:51.886648 139951474530176 tpu_context.py:202] _TPUContext: eval_on_tpu True


In [0]:
# Train the model
def model_train(estimator):
  train_features = run_classifier.convert_examples_to_features(
      train, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))



In [10]:
model_train(estimator_from_tfhub)

INFO:tensorflow:Writing example 0 of 55514


I0404 13:08:51.922581 139951474530176 run_classifier.py:774] Writing example 0 of 55514


INFO:tensorflow:*** Example ***


I0404 13:08:51.931263 139951474530176 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: train-38137


...

INFO:tensorflow:Skipping training since max_steps has already saved.


I0404 13:09:36.589204 139951474530176 estimator.py:351] Skipping training since max_steps has already saved.


INFO:tensorflow:training_loop marked as finished


I0404 13:09:36.591383 139951474530176 error_handling.py:93] training_loop marked as finished


***** Finished training at 2019-04-04 13:09:36.600614 *****


In [0]:
def model_eval(estimator):
  # Eval the model.
  eval_features = run_classifier.convert_examples_to_features(
      test, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(test)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(test) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
  output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  with tf.gfile.GFile(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
      print('  {} = {}'.format(key, str(result[key])))
      writer.write("%s = %s\n" % (key, str(result[key])))

In [12]:
model_eval(estimator_from_tfhub)

INFO:tensorflow:Writing example 0 of 13879


I0404 13:09:37.989715 139951474530176 run_classifier.py:774] Writing example 0 of 13879


INFO:tensorflow:*** Example ***


I0404 13:09:37.999026 139951474530176 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: test-67687


...

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5204: gs://stevan-bucket/bert-tfhub/models/donesi/model.ckpt-5204


I0404 13:12:16.065242 139951474530176 estimator.py:2039] Saving 'checkpoint_path' summary for global step 5204: gs://stevan-bucket/bert-tfhub/models/donesi/model.ckpt-5204


INFO:tensorflow:evaluation_loop marked as finished


I0404 13:12:18.272228 139951474530176 error_handling.py:93] evaluation_loop marked as finished


***** Finished evaluation at 2019-04-04 13:12:18.278043 *****
***** Eval results *****
  eval_accuracy = 0.94175315
  eval_loss = 0.26556543
  global_step = 5204
  loss = 0.31111827


In [0]:
def model_predict(estimator):
  # Make predictions on a subset of eval examples
  prediction_examples = test
  input_features = run_classifier.convert_examples_to_features(prediction_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
  predictions = estimator.predict(predict_input_fn)

  
  output_result_file = os.path.join(OUTPUT_DIR, "results.csv")
  with tf.gfile.GFile(output_result_file, "w") as writer:
    for example, prediction in zip(prediction_examples, predictions):
      writer.write("%s\t%s\t%s\n" % (example.text_a, str(example.label), prediction['probabilities']))

In [15]:
model_predict(estimator_from_tfhub)

INFO:tensorflow:Writing example 0 of 13879


I0404 13:12:40.078147 139951474530176 run_classifier.py:774] Writing example 0 of 13879


INFO:tensorflow:*** Example ***


I0404 13:12:40.084366 139951474530176 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: test-67687


...

I0404 13:14:32.404704 139951474530176 tpu_estimator.py:536] Enqueue next (1) batch(es) of data to infeed.


INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.


I0404 13:14:32.407005 139951474530176 tpu_estimator.py:540] Dequeue next (1) batch(es) of data from outfeed.


INFO:tensorflow:prediction_loop marked as finished


I0404 13:14:33.794270 139951474530176 error_handling.py:93] prediction_loop marked as finished


              precision    recall  f1-score   support

         0.0       0.76      0.70      0.73      1535
         1.0       0.96      0.97      0.97     12343

    micro avg       0.94      0.94      0.94     13878
    macro avg       0.86      0.83      0.85     13878
    weighted avg    0.94      0.94      0.94     13878